## Documentation
https://spark.apache.org/docs/latest/sql-data-sources-json.html

### Reading the JSON using spark sql
spark.sql("CREATE OR REPLACE TEMPORARY VIEW BABYNAME USING json OPTIONS" + 
      " (path '/data/informatica/shared/babynames/Baby.json',multiline true)")
spark.sql("select data,meta.view.id from BABYNAME").show()

### Print the schema
baby.printSchema()

val babydf2 = spark.sql("SELECT data FROM babyname")
babydf2.show()

In [ ]:
import spark.implicits._

val path = "/data/informatica/shared/babynames/Baby.json"
val baby = spark.read.option("multiline","true").json(path)
//baby.createOrReplaceTempView("babyname1")

In [ ]:
val babyname2 = baby.select( explode($"data").as("DATA1") )
//babyname2.printSchema()
//babyname2.show()

val parsed_Names = babyname2.withColumn("sid",$"DATA1"(0)).withColumn("id",$"DATA1"(1)).withColumn("position",$"DATA1"(2)).withColumn("created_at",$"DATA1"(3)).withColumn("created_meta",$"DATA1"(4)).withColumn("updated_at",$"DATA1"(5)).withColumn("updated_meta",$"DATA1"(6)).withColumn("meta",$"DATA1"(7)).withColumn("Year",$"DATA1"(8)).withColumn("first_name",$"DATA1"(9)).withColumn("county",$"DATA1"(10)).withColumn("sex",$"DATA1"(11)).withColumn("count",$"DATA1"(12)).select("sid","id", "position", "created_at", "created_meta", "updated_at", "updated_meta", "meta", "year", "first_name", "county", "sex", "count")
//parsed_Names.show(1)
parsed_Names.createOrReplaceTempView("babyname3")


In [ ]:
//parsed_Names.groupBy("year","first name").count().orderBy(asc("year"),desc("count")).show

val Pupular_Names = spark.sql("select year, first_name from (select year, first_name, count(*) as cnt,row_number() over (partition by year order by count(*) desc) as seqnum from babyname3 group by year, first_name) yn where seqnum = 1 order by year desc")
Pupular_Names.show()

//val babydf3 = spark.sql("SELECT first_name, count(first_name) as Max_Count, year FROM babyname3 group by year, First_Name order by year desc, count(First_Name) desc")
//babydf3.show()
